<h1>UST Reddit Projecy</h1></r>
<h2>Pipeline</h2>

In [ ]:
# gives accessibility to my goodle drive to fetch the data
from google.colab import drive
drive.mount('/content/drive')

<h2>Generate dataset</h2>

In [1]:
import pandas as pd
import numpy as np
import time
import re
import pickle

from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm.notebook import tqdm

<h3>Read the raw data WITHOUT CLEANING</h3>

In [ ]:
fp = '/Users/victor/Desktop/CS/UST_summer/all_posts_clean_1.csv' # the directory of the raw data file, pipeliend from reddits

s_columns = ['id', 'author', 'created_utc', 'num_comments', 'clean_title', 'clean_selftext', 'num_title_emojis', 'num_text_emojis', 'title_length', 'text_length', 'score', 'num_replied_comments', 'num_replies_by_new', 'list_new_repliers', 'num_replies_by_old', 'list_old_repliers']

df = pd.read_csv(fp, low_memory=False, encoding='utf-8')[s_columns]

df.head() # test if it is read correctly

In [ ]:
# df = df[df.created_utc < 1590000000.0]

In [ ]:
start_time = time.time()
df.groupby('author').cumcount()
print("--- %s seconds ---" % (time.time() - start_time))

df["num_post"] = df.groupby('author').cumcount().to_numpy()
codes, uniques = pd.factorize(df["author"])
df["author_codes"] = codes

# df.to_csv('wsb_with_authorcode_num_post.csv')

<h2>for statistical tests ONLY</h2>

In [ ]:
indices = df.index[(df.created_utc >= 1609459200.0) & (df.created_utc <= 1612137599.0)].tolist()

# zeros = np.zeros(df.shape[0])
# for i in indices : zeros[i] = 1

# df['is_2021Jan'] = zeros


In [ ]:
number_of_replies = list()

index = df.index[(df['clean_selftext'] == 'deleted') | (df['clean_selftext'] == 'removed')].tolist()
for i in index:
    number_of_replies.append(df['num_comments'].iloc[i])
    
# len(number_of_replies)

nonzero_replies = np.count_nonzero(np.array(number_of_replies))
nonzero_replies

In [ ]:
print(df.shape)

In [ ]:
# count the number of deleted and removed comments in each month from 2020-01-01 to 2021-05-01
date = datetime(2020,1,1,0)
time = []

for i in range(17):
    count_time = date + relativedelta(months=i)
    count_time = count_time.timestamp()
    time.append(count_time)

time


In [ ]:
def deleted_comments(left_bound, right_bound, dataframe):
    """_summary_

    Args:
        left_bound (_type_): _description_
        right_bound (_type_): _description_
        dataframe (_type_): _description_
    """
    indices = dataframe.index[(dataframe.created_utc >= left_bound) & (dataframe.created_utc <= right_bound)].tolist() # workout the total number of comments
    
    # print(indices)
    if len(indices) == 0:
        return 0
    
    unwanted_comments = 0
    for i in indices:
        if (dataframe['clean_selftext'].iloc[i] == 'deleted') | (dataframe['clean_selftext'].iloc[i] == 'removed'):
            unwanted_comments += 1
    
    return (unwanted_comments/len(indices))*100

In [ ]:
# workout the percentage of deleted commenets within the time range
unwanted_percent = []

for i in range (len(time)-1):
    left = time[i]
    right = time[i+1]
    
    
    unwanted_percent.append(deleted_comments(left,right,df))
    

In [ ]:
def replied_percent(left_bound, right_bound, dataframe):
    """_summary_

    Args:
        left_bound (_type_): _description_
        right_bound (_type_): _description_
        dataframe (_type_): _description_
    """
    indices = dataframe.index[(dataframe.created_utc >= left_bound) & (dataframe.created_utc <= right_bound)].tolist()
    
    negative_index = []
    negative_positive = 0
    
    for i in indices:
        if (dataframe['clean_selftext'].iloc[i] == 'deleted') | (dataframe['clean_selftext'].iloc[i] == 'removed'):
            negative_index.append(i)
    
    for i in negative_index:
        if dataframe.num_comments.iloc[i] > 0:
            negative_positive += 1
    
    negative_positive_rate = negative_positive / len(negative_index)
    
    positive_index = list(set(indices) - set(negative_index))
    positive_positive = 0
    
    for i in positive_index:
        if dataframe.num_comments.iloc[i] > 0:
            positive_positive += 1
    
    positive_positive_rate = positive_positive / len(positive_index)
    
    return (positive_positive_rate, negative_positive_rate)
        

In [ ]:
# workout the percentage of undeleted commenets got replied and deleted comments got replied within the time range
reply_percent = []

for i in range (len(time)-1):
    left = time[i]
    right = time[i+1]
    
    
    reply_percent.append(replied_percent(left,right,df))

In [ ]:
print(reply_percent)

In [ ]:
def AinB (A, B):
    check =  any(item in A for item in B)
    return check

def del_bot(name):
    tokens = re.split('[^a-zA-Z]', name)
    
    not_list = ['not', 'Not', 'NOt', 'NOT', 'nOt', 'nOT', 'noT', 'NoT']
    
    bot_list = ['bot', 'Bot', 'BOt', 'BOT', 'bOt', 'bOT', 'boT', 'BoT']
    
    if AinB(not_list, tokens) and AinB(bot_list, tokens):
        return False

    elif AinB(bot_list, tokens):
        return True
    
    return False

In [ ]:
# create a copy of the dataframe
botless_df = df.copy()

drop_indices = []

for i in range (len(botless_df['author'])):
    if del_bot(botless_df.author.iloc[i]):
        drop_indices.append(i)
drop_indices

In [ ]:
botless_df = botless_df.loc[~botless_df.index.isin(drop_indices)]

In [ ]:
# workout the percentage of deleted commenets within the time range
botless_unwanted_percent = []

for i in range (len(time)-1):
    left = time[i]
    right = time[i+1]
    
    
    botless_unwanted_percent.append(deleted_comments(left,right,botless_df))

In [ ]:
botless_unwanted_percent

In [ ]:
# workout the percentage of undeleted commenets got replied and deleted comments got replied within the time range
botless_reply_percent = []

for i in range (len(time)-1):
    left = time[i]
    right = time[i+1]
    
    
    botless_reply_percent.append(replied_percent(left,right,botless_df))

In [ ]:
botless_reply_percent

### **Formal Workflow**

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import re
import networkx as nx
from tqdm.notebook import tqdm
tqdm.pandas()

<h3>Clean the MONTH dataframe</h3>

In [ ]:
"""
Auxilary functions to remove bot in the raw dataframe
"""
def AinB (A, B):
    check =  any(item in A for item in B)
    return check

def del_bot(name):
    tokens = re.split('[^a-zA-Z]', name)
    
    not_list = ['not', 'Not', 'NOt', 'NOT', 'nOt', 'nOT', 'noT', 'NoT']
    
    bot_list = ['bot', 'Bot', 'BOt', 'BOT', 'bOt', 'bOT', 'boT', 'BoT']
    
    if AinB(not_list, tokens) and AinB(bot_list, tokens):
        return False

    elif AinB(bot_list, tokens):
        return True
    
    return False

In [5]:
def delBot(name):
    tokens = re.split('[^a-zA-Z]', name)
    
    not_list = ['not', 'Not', 'NOt', 'NOT', 'nOt', 'nOT', 'noT', 'NoT']
    
    bot_list = ['bot', 'Bot', 'BOt', 'BOT', 'bOt', 'bOT', 'boT', 'BoT']
    
    if re.search('\b{[not_list]}\b', name) and re.search('\b{[bot_list]}\b', name):
        return False
    elif re.search('\b{[bot_list]}\b', name):
        return True
    
    return False

In [7]:
name = "bot"

delBot(name)

False

In [ ]:
def create_name_id_dict(dataframe):
    """
    A function that is used to create a name:id dictionary
    Args:
        dataframe (pandas.dataframe): the dataframe to be exmained

    Returns:
        name_id: a dictionary having the id as the key, its corresponding name as value
    """
    name_id = {}
    for i in range(dataframe.shape[0]):
        key = dataframe.id.iloc[i]
        value = dataframe.author.iloc[i]

        key_value_pair = {key:value}

        name_id.update(key_value_pair)
    return name_id

def clean_by_parent(dataframe, name_id):
    """
    A function that is used to clean any identified BOT from the parent_id in a dataframe
    Args:
        dataframe (pandas.dataframe): the dataframe to be exmained
        name_id (dictionary): a dictionary having the id as the key, its corresponding name as value

    Returns:
        dataframe: the dataframe that is cleaned by parent_id
    """
    drop_indices = []

    for i in range(dataframe.shape[0]):
        uid = dataframe.parent_id.iloc[i].split('_')[-1]
        # use the dictionary to find the corresponding author name
        try:
            author_name = name_id[uid]
        except:
            # this would happen if the poster created the post in posterior months
            continue
        if del_bot(author_name):
            drop_indices.append(i)
    
    dataframe = dataframe.drop(drop_indices)
    
    return dataframe.reset_index(drop=True)

def clean_author(dataframe):
    """
    A function that is used to clean any identified BOT from the author in a dataframe
    Args:
        dataframe (pandas.dataframe): the dataframe to be exmained
    Returns:
        dataframe: the dataframe that is cleaned by author name
    """
    drop_indices = []

    for i in range(dataframe.shape[0]):
        author_name = dataframe.author.iloc[i]

        if del_bot(author_name):
            drop_indices.append(i)
    
    dataframe = dataframe.drop(drop_indices)

    return dataframe.reset_index(drop=True)

In [ ]:
"""
A compact function that does the cleaning for a raw dataframe
"""
def clean_df(dataframe):
    name_id = create_name_id_dict(dataframe)

    dataframe = clean_by_parent(dataframe, name_id)

    dataframe = clean_author(dataframe)

    return dataframe

-------------------------------------------


<h3>Create a list of DAY dataframe from a MONTH dataframe</h3>

In [ ]:
def workout_time(year, month):
  """
  A function that works out the timestamps of each day in a month
  Args:
      year (int): the year
      month (int): the month

  Returns:
      day_list: the timestamp of each day in a month
  """
  max_day = 31
  month_list = [4, 6, 9, 11]

  if year == 2020 and month == 2:
    max_day = 29
  elif month == 2:
    max_day = 28
  elif month in month_list:
    max_day = 30
  
  start_date = datetime(year, month, 1, 0) # always start at the first day on each month

  day_list = []

  for i in range(max_day):
    count_time = start_date + relativedelta(days=i)
    count_time = count_time.timestamp()
    day_list.append(count_time)
  
  return day_list

In [ ]:
# A compact function that works out a list of DAY dataframe from a MONTH dataframe
def create_subframe(year, month, dataframe):
  """
  A funtion that creates sub_dataframes from the parent dataframe based on number of days
  Args:
      year (int): the year
      month (int): the month
      dataframe(pandas.dataframe): the MONTH dataframe we are examine

  Returns:
      df_list: a list of DAY dataframe
  """
  df_list = []

  day_list = workout_time(year, month)

  for i in range(len(day_list)-1):
    left = day_list[i]
    right = day_list[i+1]

    day_df = dataframe[(dataframe.created_utc >= left) & (dataframe.created_utc < right)]
    
    df_list.append(day_df)
  
  return df_list

------------
<h3>Create the OUTPUT dataframe for each DAY dataframe</h3>

In [ ]:
"""
Auxilary functions to create an OUTPUT dataframe
"""
def create_output_df(daydf):
    """
    A function that creates an output dataframe for us to create the edge list later
    Args:
        daydf (pandas.dataframe): a DAY dataframe to be examined

    Returns:
        main_authors: a list of authors the gives the main post
        output_df: the output dataframe we want
    """
    tqdm.pandas()
    level_1_comments = daydf[daydf['parent_id'] == daydf['link_id']]
    main_authors = level_1_comments['author'].tolist()
    sub_comments = daydf[~daydf.index.isin(level_1_comments.index)] #setting the sub comments being those with index that are not in level_1_comments

    output_df  = sub_comments.groupby('parent_id')['author'].apply(list).reset_index(name='author')
    output_df['sub_comments'] = output_df['author'].progress_apply(lambda x:len(x))
    output_df.sort_values('sub_comments', inplace=True, ascending=False)

    return main_authors, output_df

def is_main(outputdf, daydf):
    """
    A function that check if a post from the OUTPUT dataframe is recognised as a MAIN post
    Args:
        outputdf (): the OUTPUT dataframe
        daydf (): the DAY dataframe

    Returns:
        outputdf: with two more columns. One is 'main_post': where 1=main post; 0 otherwise
                                         The other is 'parent_author', gives the author name of the corresponding parent id
    """
    main_list = []
    parent_author_list = []

    for i in range(outputdf.shape[0]):
        uid = outputdf.parent_id[i].split('_')[-1]
        ind = daydf[daydf['id'] == uid].index.to_list()
        # print(ind)
        if ind != []:
            parent_author_list.append(daydf.loc[ind[0], 'author'])
            if daydf.loc[ind[0], 'parent_id'] == daydf.loc[ind[0], 'link_id']:
                main_list.append(1)
            else:
                main_list.append(0)
        else:
            main_list.append(np.nan)
            parent_author_list.append(np.nan)
            
    outputdf['main_post'] =  main_list
    outputdf['parent_author'] = parent_author_list
    
    return outputdf

In [ ]:
# A compact function that works out the OUTPUT dataframe
def output_dataframe(daydf):
    """
    A function that works out the OUTPUT dataframe from a DAY dataframe.
    5 columns: parent_id, author, sub_comments, main_post and parent_author
    Args:
        daydf (): the DAY dataframe

    Returns:
        outputdf: the OUTPUT dataframe that is being reset index
    """
    main_authors, outputdf = create_output_df(daydf)

    outputdf = is_main(outputdf, daydf)

    return main_authors, outputdf.reset_index(drop=True)

-------------------------------
<h3>Create the adjacency matrix for a OUTPUT dataframe</h3>

In [ ]:
"""
Auxilary functions to create the adj matrix
"""
def get_all_authors(name_id_day, output_df):
    """
    A function to get all author names who created the post on this DAY from a OUTPUT dataframe 
    Args:
        name_id_day (dictionary): a dictionary having the id as the key, its corresponding name as value from a DAY dataframe
        output_df (pandas.dataframe): the output dataframe to be examined

    Returns:
        author_list: a list of author names
    """

    author_list = set()
    for i in range(output_df.shape[0]):
        uid = output_df.parent_id.iloc[i].split('_')[-1]

        try:
            author_list.add(name_id_day[uid])
        except:
            # this would happend if author created the post on a posterior day
            continue

        author_list.update(tuple(output_df.author.iloc[i]))
        
    return author_list

In [ ]:
# A compact function that returns the adj matrix
def generate_adj_matrix(outputdf, name_id_day): 
    """
    A function to generate the adjacency matrix
    Args:
        outputdf (): the OUTPUT dataframe of the corresponding DAY dataframe
        name_id_day (dictionary): a dictionary having the id as the key, its corresponding name as value from a DAY dataframe

    Returns:
        adj_matrix: the matrix
    """
    # create the dict from the DAY dataframe
    author_list = get_all_authors(name_id_day, outputdf)

    adj_matrix = pd.DataFrame(0, index = author_list, columns = author_list)

    for i in range(outputdf.shape[0]):
        uid = outputdf.parent_id.iloc[i].split('_')[-1]
        
        try:
            parent_name = name_id_day[uid]
        except:
            continue
        author_list = outputdf.author.iloc[i]

        for author in author_list:
            adj_matrix.loc[author, parent_name] += 1
    
    return adj_matrix

---------------
<h3>Create an edge list from a DAY dataframe</h3>

In [ ]:
def get_edge_list(outputdf, name_id_day, main_authors): 
    """
    A function to generate edge list dataframe
    Args:
        outputdf (): the OUTPUT dataframe
        name_id_day (dictionary): a dictionary having the id as the key, its corresponding name as value from a DAY dataframe
        main_authors (list): a list of authors the gives the main post

    Returns:
        edge_list: a dataframe of edge list with 3 colums, each are Source, Target and is_main
    """
    source_list = []
    target_list = []
    is_main_list = []
    for i in range(outputdf.shape[0]):
        uid = outputdf.parent_id.iloc[i].split('_')[-1]        
        try:
            parent_name = name_id_day[uid]
            main_post = 1 if parent_name in main_authors else  0 
            author_list = outputdf.author.iloc[i]
            for author in author_list:
                source_list.append(author)
                target_list.append(parent_name)
                is_main_list.append(main_post)
        except:
            continue
    edge_list = pd.DataFrame(list(zip(source_list, target_list, is_main_list)), columns = ['Source', 'Target', 'is_main'])

    return edge_list

def get_singleton(outputdf, daydf):
    """
    A function that works out a list of authors recognised as singleton nodes in the edge list
    Args:
        outputdf (): the OUTPUT dataframe 
        daydf (): the DAY dataframe 

    Returns:
        singleton: a list of author name that are singleton
    """
    
    complete_authors = daydf['author'].to_list()
    non_single = []
    for i in range(outputdf.shape[0]):
        if outputdf.parent_author.iloc[i] == np.nan:
            continue
        else:
            non_single.append(outputdf.parent_author.iloc[i])
            non_single += outputdf.author.iloc[i]
    non_single = list(dict.fromkeys(non_single))
    complete_authors = list(dict.fromkeys(complete_authors))
    singleton = set(complete_authors) - set(non_single)
    
    return singleton

def singleton_edge(edge_list, singleton):
    """
    A function adds singleton nodes to an existing edge list
    Args:
        edge_list (): the edge list
        singleton (list):  a list of author name that are singleton

    Returns:
        edge_list: the updated edge list where SOURCE=TARGET=author from singleton and WEIGHT = 1
    """
    
    for author in singleton:
        row = [author, author, 1]
        edge_list.loc[len(edge_list.index)] = row
    
    return edge_list

In [ ]:
# A compact function that returns two different edge list from an OUTPUT dataframe
def e_lists(outputdf, daydf, main_authors, name_id_day):
    standard_e = get_edge_list(outputdf, name_id_day, main_authors)

    # to return the updated edge list
    single = get_singleton(outputdf, daydf)

    updated_e = standard_e[['Source', 'Target']]
    updated_e = updated_e.groupby(updated_e.columns.tolist()).size().reset_index().rename(columns={0:'weights'})

    updated_e = singleton_edge(updated_e, single)

    return standard_e, updated_e

----------------
<h2>To walk throught a month</h2>

In [ ]:
# a CHEAP month walker by now
def month_walker(raw_df, year, month):
    """
    A function returns a list of DAY information from a DAY dataframe inside a list of DAY-frames
    The information has: 1. OUTPUT dataframe, 2. Adjacency matrix and 3. Two types of edge lists
    Each information is of type tuple
    Args:
        raw_df (): dataframe not cleaned
        year (int): the year
        month (int): the month

    Returns:
        month_info: a list of information
    """
    # clean dataframe
    raw_df = raw_df.reset_index(drop=True)
    cleaned_df = clean_df(raw_df)
    # create a list of DAY dataframes
    df_list = create_subframe(year, month, cleaned_df)

    month_info = []

    for daydf in df_list:
        # this gives the name id dictionary for TODAY
        name_id_day = create_name_id_dict(daydf)
        # this gives you the OUTPUT dataframe
        main_authors, outputdf = output_dataframe(daydf)
        # this gives the adj matrix
        adj_mat = generate_adj_matrix(outputdf, name_id_day)
        # this gives the two edge lists
        standard_e, updated_e = e_lists(outputdf, daydf, main_authors, name_id_day)

        day_info = (outputdf, adj_mat, standard_e, updated_e)

        month_info.append(day_info)
    
    return month_info

TEST

In [ ]:
fp1 = "/home/vw/Desktop/UST_summer/01-20/01-20_comments.csv"
df_01 = pd.read_csv(fp1, low_memory = False, encoding = 'utf-8')  #get the Jan-2020 spreadsheet
df_01 = df_01[df_01.author != '[deleted]'] #remove authors that are deleted

In [ ]:
month, year = 1, 2020
raw_df = df_01

In [ ]:
jan_2020_info = month_walker(raw_df,year,month)

Testing

In [2]:
post_authors = pd.read_csv('/Users/victor/Desktop/CS/UST_summer/posts_authors.csv')

In [3]:
with open('/Users/victor/Desktop/CS/UST_summer/main_authors.pickle', 'rb') as handle:
    main_authors = pickle.load(handle)

In [4]:
with open('/Users/victor/Desktop/CS/UST_summer/name_id.pickle', 'rb') as handle:
    name_id = pickle.load(handle)

In [5]:
sliced_post = post_authors[:100000]

In [6]:
def is_main(dataframe, name_id_dictionary, authorlist):
    main_list = []
    parent_author_list = []
    dataframe = dataframe.reset_index(drop = True)
    for i in range(dataframe.shape[0]):
        uid = dataframe.parent_id[i].split('_')[-1]
        
        try:
            parent_author = name_id_dictionary[uid]
            parent_author_list.append(parent_author)
            
            if parent_author in authorlist:
                main_list.append(1)
            else:
                main_list.append(0)
        except:
            main_list.append(np.nan)
            parent_author_list.append(np.nan)
            
        try:
            if i % 100000 == 0:
                main_filename = 'main_list'+str(i)+'.pickle'
                post_filename = 'post_list'+str(i)+'.pickle'
                with open(main_filename, 'wb') as handle:
                    pickle.dump(main_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
                with open(post_filename, 'wb') as handle:
                    pickle.dump(parent_author_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
        except:
            print('kernel broken at i = ' + str(i))
    dataframe['main_post'] =  main_list
    dataframe['parent_author'] = parent_author_list
    return dataframe

In [7]:
new_post = is_main(sliced_post, name_id, main_authors)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

----------------------

In [ ]:
# def month_walker(month_df, path, mm_yy, save=True):
#     if save:
#         folder_path = os.path.join(path, mm_yy)
#         os.makedirs(folder_path) 
#     for day_df in month_df:
#         i = 1
#         # first we create a pc_df and a dictionary to store the name
#         main_authors, posts_authors = create_parent_children_df(day_df)
#         name_id = create_name_id_dict(day_df)
#         # then we clean pc_df by parent_id and its author column
#         posts_authors = posts_authors.drop(clean_by_parent(posts_authors, name_id))
#         posts_authors = clean_author(posts_authors)
#         # finally create the adj matrix
#         author_list = get_all_authors(name_id, posts_authors)
#         source_list = []
#         target_list = []
#         is_main_list = []
#         for i in range(dataframe.shape[0]):
#           uid = dataframe.parent_id.iloc[i].split('_')[-1]        
#           try:
#             parent_name = name_id_dict[uid]
#             main_post = 1 if parent_name in main_authors else  0 
#             author_list = dataframe.author.iloc[i]
#             for author in author_list:
#               source_list.append(author)
#               target_list.append(parent_name)
#               is_main_list.append(main_post)
#           except:
#             continue
#         edge_list = pd.DataFrame(list(zip(source_list, target_list, is_main_list)), columns = ['Source', 'Target', 'is_main'])
#         # store the matrix as pickle
#         save_path = os.path.join(folder_path, str(i))
#         edge_list.to_pickle(save_path)
#         i += 1
    
#     return matrices